In [1]:
# Import necessary library for the program
import sqlite3

# Connect to the SQLite database
con = sqlite3.connect("CarSharing.db", isolation_level=None)
cur = con.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()
for table in tables:
    cur.execute("DROP TABLE %s" %table)

# Create a table for the "CarSharing" csv file
cur.execute('''CREATE TABLE carsharing (
                    id INT NOT NULL,
                    timestamp DATETIME,
                    season VARCHAR(8),
                    holiday VARCHAR(5),
                    workingday VARCHAR(5),
                    weather VARCHAR(35),
                    temp DECIMAL(2),
                    temp_feel DECIMAL(3),
                    humidity INT,
                    windspeed DECIMAL(4),
                    demand DECIMAL(9),
                    UNIQUE(id)
                    )''')

# Read data from the CSV fil
with open('CarSharing.csv', 'r') as file:
    lines = file.readlines()

list = []

# Process each line of the CSV file
for line in lines[1:]:
    # Split the line into values
    values = line.strip().split(',')
    # Split the string into date and time components
    date_str, time_str = values[1].split()
    # Split the date into day, month, and year components
    day, month, year = date_str.split("/")
    # Rearrange the components to form the desired format
    values[1] = f"{year}-{month}-{day} {time_str}"
    
    cur.execute('''INSERT INTO CarSharing VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);''', values)

attributes = ['temp', 'temp_feel', 'humidity', 'windspeed', 'demand']

for attribute in attributes:
    sql = f"UPDATE CarSharing SET {attribute} = NULL WHERE {attribute} = ''"
    cur.execute(sql)

# Create a backup table
cur.execute("CREATE TABLE BackUp AS SELECT * FROM CarSharing;")

# Retrieve and print data from the "CarSharing" table
cur.execute("SELECT * From CarSharing;")
result=cur.fetchall()

# Write data to the CSV file
with open("Result1a.csv", 'w') as file:
    # Write the column headers to the CSV file
    column_names = [desc[0] for desc in cur.description]
    file.write(','.join(column_names) + '\n')
    
    # Write the data rows to the CSV file
    for rows in result:
        file.write(','.join(str(value) for value in rows) + '\n')

# Retrieve and print data from the "Backup" table
cur.execute("SELECT * From Backup;")
result = cur.fetchall()

# Write data to the CSV file
with open("Result1b.csv", 'w') as file:
    # Write the column headers to the CSV file
    column_names = [desc[0] for desc in cur.description]
    file.write(','.join(column_names) + '\n')
    
    # Write the data rows to the CSV file
    for rows in result:
        file.write(','.join(str(value) for value in rows) + '\n')

# Close the connection
cur.close()
print("Completed!")

Completed!


In [2]:
# Connect to the SQLite database
cur = con.cursor()

# cur.execute("ALTER TABLE CarSharing DROP temp_category")

# Add temp_category column
cur.execute('''ALTER TABLE CarSharing
ADD temp_category varchar(5);''')

# Update temp_category column based on temp_feel
cur.execute('''UPDATE CarSharing
SET temp_category =
CASE
    WHEN temp_feel is NULL THEN NULL
    WHEN temp_feel < 10 THEN "Cold"
    WHEN temp_feel > 25 THEN "Hot"
    ELSE "Mild"
END;''')

# Retrieve data from the CarSharing table
cur.execute("SELECT * From CarSharing;")
result = cur.fetchall()

# Write data to the CSV file
with open("Result2.csv", 'w') as file:
    # Write the column headers to the CSV file
    column_names = [desc[0] for desc in cur.description]
    file.write(','.join(column_names) + '\n')
    
    # Write the data rows to the CSV file
    for rows in result:
        file.write(','.join(str(value) for value in rows) + '\n')

# Close the connection
cur.close()
print("Completed!")

Completed!


In [3]:
cur = con.cursor()

# cur.execute("DROP TABLE temperature")

cur.execute("CREATE TABLE temperature AS SELECT temp, temp_feel, temp_category FROM CarSharing;")

cur.execute("""ALTER TABLE CarSharing
DROP COLUMN temp;""")
cur.execute("""ALTER TABLE CarSharing
DROP COLUMN temp_feel;""")

cur.execute("SELECT * From temperature;")
result = cur.fetchall()
# Write data to the CSV file
with open("Result3a.csv", 'w') as file:
    # Write the column headers to the CSV file
    column_names = [desc[0] for desc in cur.description]
    file.write(','.join(column_names) + '\n')
    
    # Write the data rows to the CSV file
    for rows in result:
        file.write(','.join(str(value) for value in rows) + '\n')

cur.execute("SELECT * From CarSharing;")
result = cur.fetchall()
# Write data to the CSV file
with open("Result3b.csv", 'w') as file:
    # Write the column headers to the CSV file
    column_names = [desc[0] for desc in cur.description]
    file.write(','.join(column_names) + '\n')
    
    # Write the data rows to the CSV file
    for rows in result:
        file.write(','.join(str(value) for value in rows) + '\n')

# Close the connection
cur.close()
print("Completed!")

Completed!


In [4]:
cur = con.cursor()

cur.execute("SELECT DISTINCT(weather) From CarSharing;")
result = cur.fetchall()
# print(result)

dictA = {}
varA = 1
for x in result:
    for y in x:
        dictA[y] = varA
    varA += 1

# print(dictA)

#cur.execute("ALTER TABLE Raw DROP weather_code")

cur.execute('''ALTER TABLE CarSharing
ADD weather_code int;''')

for weather, code in dictA.items():
    cur.execute("UPDATE CarSharing SET weather_code = ? WHERE weather = ?;", (code, weather))

cur.execute("SELECT * From CarSharing;")
result = cur.fetchall()
# Write data to the CSV file
with open("Result4.csv", 'w') as file:
    # Write the column headers to the CSV file
    column_names = [desc[0] for desc in cur.description]
    file.write(','.join(column_names) + '\n')
    
    # Write the data rows to the CSV file
    for rows in result:
        file.write(','.join(str(value) for value in rows) + '\n')

# Close the connection
cur.close()
print("Completed!")

Completed!


In [5]:
cur = con.cursor()

# cur.execute("DROP TABLE weather")

cur.execute("CREATE TABLE weather AS SELECT weather, weather_code FROM CarSharing;")

cur.execute("""ALTER TABLE CarSharing
DROP COLUMN weather;""")

cur.execute("SELECT * From weather;")
result = cur.fetchall()
# Write data to the CSV file
with open("Result5a.csv", 'w') as file:
    # Write the column headers to the CSV file
    column_names = [desc[0] for desc in cur.description]
    file.write(','.join(column_names) + '\n')
    
    # Write the data rows to the CSV file
    for rows in result:
        file.write(','.join(str(value) for value in rows) + '\n')

cur.execute("SELECT * From CarSharing;")
result = cur.fetchall()
# Write data to the CSV file
with open("Result5b.csv", 'w') as file:
    # Write the column headers to the CSV file
    column_names = [desc[0] for desc in cur.description]
    file.write(','.join(column_names) + '\n')
    
    # Write the data rows to the CSV file
    for rows in result:
        file.write(','.join(str(value) for value in rows) + '\n')

# Close the connection
cur.close()
print("Completed!")

Completed!


In [6]:
cur = con.cursor()

# cur.execute("DROP TABLE time;")

cur.execute("""CREATE TABLE time (
    timestamp DATETIME,
    hour INT,
    weekday INT,
    month INT);""")

cur.execute("""INSERT INTO time SELECT timestamp, strftime("%H", timestamp) AS hour, strftime("%w", timestamp) AS weekday, strftime("%m", timestamp) AS month From CarSharing;""")

weekdays = { 0: "Sunday", 1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday"}
for code, name in weekdays.items():
    cur.execute("UPDATE time SET weekday = ? WHERE weekday = ?;", (name, code))

months = { 1: "January", 2: "February", 3: "March", 4: "April", 5: "May", 6: "June", 7: "July", 8: "August", 9: "September", 10: "October", 11: "November", 12: "December"}
for code, name in months.items():
    cur.execute("UPDATE time SET month = ? WHERE month = ?;", (name, code))

cur.execute("SELECT * From time;")
result = cur.fetchall()
# Write data to the CSV file
with open("Result6.csv", 'w') as file:
    # Write the column headers to the CSV file
    column_names = [desc[0] for desc in cur.description]
    file.write(','.join(column_names) + '\n')
    
    # Write the data rows to the CSV file
    for rows in result:
        file.write(','.join(str(value) for value in rows) + '\n')

# Close the connection
cur.close()
print("Completed!")

Completed!


In [7]:
cur = con.cursor()

cur.execute("""
SELECT strftime("%Y-%m-%d", timestamp) AS date, strftime("%H:%M:%S", timestamp) AS time, demand
FROM CarSharing
WHERE id IN (SELECT id FROM CarSharing WHERE strftime("%Y", timestamp) = "2017")
AND demand IN (SELECT MAX(demand) FROM CarSharing WHERE strftime("%Y", timestamp) = "2017");""")

result = cur.fetchall()
columns = [description[0] for description in cur.description]
print(columns, result, sep="\n")
cur.close()

['date', 'time', 'demand']
[('2017-06-15', '17:00:00', 6.458338283)]


In [8]:
cur = con.cursor()

# cur.execute("DROP TABLE result")

cur.execute("""CREATE TABLE result AS SELECT t.weekday AS weekday, t.month AS month, c.season AS season, AVG(c.demand) AS average_demand
FROM CarSharing c
INNER JOIN time t
ON c.timestamp = t.timestamp
WHERE strftime("%Y", c.timestamp) = "2017"
GROUP BY weekday, month, season
ORDER BY average_demand DESC
LIMIT 1;""")

cur.execute("""INSERT INTO result (weekday, month, season, average_demand)
SELECT t.weekday AS weekday, t.month AS month, c.season AS season, AVG(c.demand) AS average_demand
FROM CarSharing c
INNER JOIN time t
ON c.timestamp = t.timestamp
WHERE strftime("%Y", c.timestamp) = "2017"
GROUP BY weekday, month, season
ORDER BY average_demand
LIMIT 1;""")

cur.execute("SELECT * From result;")
result = cur.fetchall()

# Write data to the CSV file
with open("Result7b.csv", 'w') as file:
    # Write the column headers to the CSV file
    column_names = [desc[0] for desc in cur.description]
    file.write(','.join(column_names) + '\n')
    
    # Write the data rows to the CSV file
    for rows in result:
        file.write(','.join(str(value) for value in rows) + '\n')

columns = [description[0] for description in cur.description]
print(columns, result, sep="\n")
cur.close()

['weekday', 'month', 'season', 'average_demand']
[('Sunday', 'July', 'fall', 4.997135078819442), ('Monday', 'January', 'spring', 3.05078577812676)]


In [9]:
cur = con.cursor()

cur.execute("""SELECT t.weekday, t.hour, AVG(c.demand) AS average_demand
FROM CarSharing c
INNER JOIN time t
ON c.timestamp = t.timestamp
WHERE strftime("%Y", c.timestamp) = "2017"
AND weekday IN (SELECT weekday FROM result)
GROUP BY t.weekday, t.hour
ORDER BY average_demand DESC;""")

result = cur.fetchall()
# Write data to the CSV file
with open("Result7c.csv", 'w') as file:
    # Write the column headers to the CSV file
    column_names = [desc[0] for desc in cur.description]
    file.write(','.join(column_names) + '\n')
    
    # Write the data rows to the CSV file
    for rows in result:
        file.write(','.join(str(value) for value in rows) + '\n')
columns = [description[0] for description in cur.description]
print(columns, result, sep="\n")
cur.close()

['weekday', 'hour', 'average_demand']
[('Monday', 13, 5.64355388559375), ('Monday', 12, 5.6219724075), ('Monday', 14, 5.5546127640312495), ('Sunday', 15, 5.537925196764707), ('Monday', 15, 5.5151148165625), ('Sunday', 14, 5.513702656235295), ('Monday', 16, 5.503753106031249), ('Sunday', 16, 5.496274498382354), ('Sunday', 13, 5.478758230294119), ('Sunday', 12, 5.457459126911766), ('Monday', 11, 5.437364716968752), ('Monday', 17, 5.39925222134375), ('Sunday', 17, 5.36763434964706), ('Sunday', 11, 5.2909150675882355), ('Sunday', 18, 5.241211627117648), ('Monday', 10, 5.22383141215625), ('Monday', 18, 5.215942911343751), ('Sunday', 10, 5.074545768735294), ('Sunday', 19, 5.041802430029413), ('Monday', 19, 4.99049988034375), ('Sunday', 20, 4.790666498058823), ('Monday', 20, 4.72698612103125), ('Sunday', 9, 4.68317637832353), ('Monday', 9, 4.63834451678125), ('Sunday', 21, 4.604221629500001), ('Sunday', 22, 4.477619972735296), ('Monday', 21, 4.464855592749998), ('Sunday', 23, 4.34675400444117

In [10]:
cur = con.cursor()

cur.execute("""SELECT temp_category, COUNT(temp_category)
FROM CarSharing
WHERE strftime("%Y", timestamp) = "2017"
GROUP BY temp_category
ORDER BY COUNT(temp_category) DESC
LIMIT 1
;""")

result = cur.fetchall()
columns = [description[0] for description in cur.description]
print(columns, result, sep="\n")

cur.close()

['temp_category', 'COUNT(temp_category)']
[('Mild', 2660)]


In [11]:
cur = con.cursor()

cur.execute("""SELECT w.weather, COUNT(CarSharing.weather_code)
FROM CarSharing
INNER JOIN (
  SELECT DISTINCT weather_code, weather
  FROM weather
) w
ON CarSharing.weather_code = w.weather_code
WHERE strftime("%Y", CarSharing.timestamp) = "2017"
GROUP BY CarSharing.weather_code
ORDER BY COUNT(CarSharing.weather_code) DESC
LIMIT 1;""")

result = cur.fetchall()
columns = [description[0] for description in cur.description]
print(columns, result, sep="\n")

cur.close()

['weather', 'COUNT(CarSharing.weather_code)']
[('Clear or partly cloudy', 3583)]


In [12]:
cur = con.cursor()

cur.execute("""SELECT t.month, AVG(c.windspeed) AS average_windspeed, MAX(c.windspeed) AS max_windspeed, MIN(c.windspeed) AS min_windspeed
FROM CarSharing c
INNER JOIN time t
ON c.timestamp = t.timestamp
WHERE strftime("%Y", c.timestamp) = "2017"
GROUP BY t.month
;""")

result = cur.fetchall()
columns = [description[0] for description in cur.description]
print(columns, result, sep="\n")

cur.execute("""SELECT t.month, AVG(c.humidity) AS average_humidity, MAX(c.humidity) AS max_humidity, MIN(c.humidity) AS min_humidity
FROM CarSharing c
INNER JOIN time t
ON c.timestamp = t.timestamp
WHERE strftime("%Y", c.timestamp) = "2017"
GROUP BY t.month
;""")

result = cur.fetchall()
columns = [description[0] for description in cur.description]
print(columns, result, sep="\n")

cur.close()

['month', 'average_windspeed', 'max_windspeed', 'min_windspeed']
[('April', 15.852275112107584, 40.9973, 0), ('August', 12.411122347629782, 43.0006, 0), ('December', 10.8364595505618, 43.0006, 0), ('February', 15.577716628175509, 51.9987, 0), ('January', 13.748052358490567, 39.0007, 0), ('July', 12.015845657015571, 56.9969, 0), ('June', 11.827618161434977, 35.0008, 0), ('March', 15.974884101382482, 40.9973, 0), ('May', 12.427390827740476, 40.9973, 0), ('November', 12.142271171171178, 36.9974, 0), ('October', 10.892052370203167, 36.9974, 0), ('September', 11.564080089485438, 40.9973, 0)]
['month', 'average_humidity', 'max_humidity', 'min_humidity']
[('April', 66.2488986784141, 100, 22), ('August', 62.17362637362638, 94, 25), ('December', 65.18061674008811, 100, 26), ('February', 53.58071748878924, 100, 8), ('January', 56.30769230769231, 100, 28), ('July', 60.29203539823009, 94, 17), ('June', 58.370860927152314, 100, 20), ('March', 55.997752808988764, 100, 0), ('May', 71.37142857142857, 

In [13]:
cur = con.cursor()

cur.execute("""SELECT temp_category, AVG(demand)
FROM CarSharing
WHERE strftime("%Y", CarSharing.timestamp) = "2017"
AND temp_category IN ("Cold", "Mild", "Hot")
GROUP BY temp_category
ORDER BY AVG(demand) DESC
;""")

result = cur.fetchall()
columns = [description[0] for description in cur.description]
print(columns, result, sep="\n")

cur.close()

['temp_category', 'AVG(demand)']
[('Hot', 4.798172530447556), ('Mild', 4.021015429114678), ('Cold', 3.1902527494948725)]


In [17]:
cur = con.cursor()

cur.execute("""SELECT t.month, AVG(c.demand) as average_demand
FROM CarSharing c
INNER JOIN time t
ON c.timestamp = t.timestamp
WHERE strftime("%Y", c.timestamp) = "2017"
GROUP BY t.month
ORDER BY average_demand DESC
LIMIT 1
;""")

result = cur.fetchall()
columns = [description[0] for description in cur.description]
print(columns, result, sep="\n")


cur.execute("""SELECT c.timestamp, c.windspeed, c.humidity, w.weather, c.temp_category
FROM CarSharing c
INNER JOIN time t
ON c.timestamp = t.timestamp
INNER JOIN (
  SELECT DISTINCT weather_code, weather
  FROM weather
) w
ON c.weather_code = w.weather_code
WHERE t.month = "July"
AND strftime("%Y", c.timestamp) = "2017"
;""")

result = cur.fetchall()
columns = [description[0] for description in cur.description]
print(columns, result, sep="\n")

cur.close()

['month', 'average_demand']
[('July', 4.787654876394732)]
['timestamp', 'windspeed', 'humidity', 'weather', 'temp_category']
[('2017-07-01 00:00', 0, 50, 'Clear or partly cloudy', 'Hot'), ('2017-07-01 01:00', 0, 50, 'Clear or partly cloudy', 'Hot'), ('2017-07-01 02:00', 0, 69, 'Clear or partly cloudy', 'Hot'), ('2017-07-01 03:00', 0, 53, 'Clear or partly cloudy', 'Hot'), ('2017-07-01 04:00', 0, 57, 'Clear or partly cloudy', 'Hot'), ('2017-07-01 05:00', 8.9981, 64, 'Clear or partly cloudy', 'Hot'), ('2017-07-01 06:00', 8.9981, 61, 'Clear or partly cloudy', 'Hot'), ('2017-07-01 07:00', 7.0015, 54, 'Clear or partly cloudy', 'Hot'), ('2017-07-01 08:00', 11.0014, 42, 'Clear or partly cloudy', 'Hot'), ('2017-07-01 09:00', 11.0014, 35, 'Clear or partly cloudy', 'Hot'), ('2017-07-01 10:00', 11.0014, 33, 'Clear or partly cloudy', 'Hot'), ('2017-07-01 11:00', 0, 26, 'Clear or partly cloudy', 'Hot'), ('2017-07-01 12:00', 0, 26, 'Clear or partly cloudy', 'Hot'), ('2017-07-01 13:00', 0, 24, 'Clear 